# Web3 client

In [2]:
from web3 import Web3
from web3.middleware import geth_poa_middleware
import json

In [3]:
LIDO_ARTIFACT = 'Lido.json'
ORACLE_ARTIFACT = 'LidoOracle.json'
NODE_OPS_REGISTRY_ARTIFACT = 'NodeOperatorsRegistry.json'
WEB3_URL = 'http://206.81.31.11/rpc'
BEACON_URL = 'http://206.81.31.11:5052/eth/v1' #Lighthouse
LIDO_ADDRESS='0xA5d26F68130c989ef3e063c9bdE33BC50a86629D'

In [4]:
with open(LIDO_ARTIFACT) as f:
    lido_artifact = json.load(f)
    
with open(ORACLE_ARTIFACT) as f:
    oracle_artifact = json.load(f)
    
with open(NODE_OPS_REGISTRY_ARTIFACT) as f:
    nops_artifact = json.load(f)

In [5]:
w3 = Web3(Web3.HTTPProvider(WEB3_URL, request_kwargs={'timeout': 60}))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [6]:
w3.eth.chainId

5

In [7]:
w3.eth.blockNumber

3860988

In [8]:
w3.clientVersion

'Geth/v1.9.24-unstable-97fc1c3b-20201105/linux-amd64/go1.15.3'

In [9]:
w3.eth.getBlock('latest')

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x476f65726c6920496e697469617469766520417574686f726974790000000000b92c1cfee0307ba74a3231961dd77796e48828b72ea7246dd2638ce9aa63f8b93837044f3da9299134329d90918b86d6594bcb8a76fe902ca9e1a7689f56476201'),
 'gasLimit': 8000000,
 'gasUsed': 0,
 'hash': HexBytes('0xf6a45b7cff36b16ed67035b107340d0239afb9dd14d5dacc02b2538b5fc1f7d8'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x0000000000000000000000000000000000000000',
 'mixHash

## Instantiate contracts

In [10]:
lido = w3.eth.contract(
    address=LIDO_ADDRESS,
    abi=lido_artifact['abi'],
)
lido.functions.getBeaconStat().call()

[43, 37, 1184760676725000000000]

In [11]:
nops_address = lido.functions.getOperators().call()
oracle_address = lido.functions.getOracle().call()
nops_address, oracle_address

('0xB1e7Fb9E9A71063ab552dDEE87Ea8C6eEc7F5c7A',
 '0x8aA931352fEdC2A5a5b3E20ed3A546414E40D86C')

In [12]:
oracle = w3.eth.contract(address=oracle_address, abi=oracle_artifact['abi'])
( epochsPerFrame, slotsPerEpoch, secondsPerSlot, genesisTime ) = oracle.functions.beaconSpec().call()
epochsPerFrame, slotsPerEpoch, secondsPerSlot, genesisTime 

(12, 32, 12, 1605700807)

In [13]:
nops = w3.eth.contract(address=nops_address, abi=nops_artifact['abi'])
nops.functions.getNodeOperatorsCount().call()

6

In [14]:
members_addresses = oracle.functions.getOracleMembers().call()

In [15]:
epoch, frame_start, frame_end = oracle.functions.getCurrentFrame().call()
epoch, frame_start, frame_end

(3348, 1606986439, 1606991046)

In [16]:
epoch = oracle.functions.getCurrentEpochId().call()
epoch

3350

In [17]:
oracle.functions.getQuorum().call()

2

In [18]:
nops.functions.getActiveNodeOperatorsCount().call()

6

In [19]:
depositedValidators, beaconValidators,beaconBalance = lido.functions.getBeaconStat().call()
depositedValidators, beaconValidators, beaconBalance

(43, 37, 1184760676725000000000)